<a href="https://colab.research.google.com/github/muhammadhafiz27/Deep-Learning/blob/main/Deep_Belief_Network_(DBN)_dan_Capsule_Network_(CapsNet).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Implementasi Deep Belief Network (DBN)**

## **Import Library**

In [1]:
from sklearn.neural_network import BernoulliRBM
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import MinMaxScaler
import numpy as np

## **Load dataset MNIST**

In [2]:
X, y = fetch_openml('mnist_784', version=1, return_X_y=True)
X = X.astype('float32')
X = MinMaxScaler().fit_transform(X)
y = y.astype('int')

## **Definisikan RBM dan Logistic Regression**

In [3]:
rbm = BernoulliRBM(n_components=256, learning_rate=0.06, batch_size=10, n_iter=10, verbose=1)
logistic = LogisticRegression(max_iter=1000, solver='lbfgs', multi_class='multinomial')

## **DBN sebagai pipeline RBM + Logistic Regression**

In [4]:
dbn = Pipeline(steps=[('rbm', rbm), ('logistic', logistic)])
dbn.fit(X[:60000], y[:60000])

[BernoulliRBM] Iteration 1, pseudo-likelihood = -86.59, time = 24.38s
[BernoulliRBM] Iteration 2, pseudo-likelihood = -85.00, time = 24.51s
[BernoulliRBM] Iteration 3, pseudo-likelihood = -81.84, time = 21.78s
[BernoulliRBM] Iteration 4, pseudo-likelihood = -81.03, time = 22.48s
[BernoulliRBM] Iteration 5, pseudo-likelihood = -82.63, time = 23.44s
[BernoulliRBM] Iteration 6, pseudo-likelihood = -82.28, time = 23.64s
[BernoulliRBM] Iteration 7, pseudo-likelihood = -83.51, time = 23.58s
[BernoulliRBM] Iteration 8, pseudo-likelihood = -83.89, time = 21.68s
[BernoulliRBM] Iteration 9, pseudo-likelihood = -78.88, time = 23.69s
[BernoulliRBM] Iteration 10, pseudo-likelihood = -78.20, time = 23.59s


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Pipeline(steps=[('rbm', BernoulliRBM(learning_rate=0.06, verbose=1)),
                ('logistic',
                 LogisticRegression(max_iter=1000, multi_class='multinomial'))])

## **Evaluasi**

In [5]:
score = dbn.score(X[60000:], y[60000:])
print(f"Akurasi DBN pada data uji: {score * 100:.2f}%")

Akurasi DBN pada data uji: 96.21%


# **Implementasi Capsule Network (CapsNet)**

## **Import Library**

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

## **Load dataset**

In [7]:
transform = transforms.Compose([transforms.ToTensor()])
train_loader = DataLoader(datasets.MNIST('./data', train=True, download=True, transform=transform), batch_size=64, shuffle=True)
test_loader = DataLoader(datasets.MNIST('./data', train=False, transform=transform), batch_size=64, shuffle=False)

100%|██████████| 9.91M/9.91M [00:00<00:00, 20.1MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 483kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.50MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 11.8MB/s]


## **Definisi CapsNet sederhana**

In [8]:
class CapsuleLayer(nn.Module):
    def __init__(self, num_capsules, num_route_nodes, in_channels, out_channels, num_iterations=3):
        super(CapsuleLayer, self).__init__()
        self.num_route_nodes = num_route_nodes
        self.num_capsules = num_capsules
        self.num_iterations = num_iterations
        self.route_weights = nn.Parameter(torch.randn(num_capsules, num_route_nodes, in_channels, out_channels))

    def squash(self, tensor, dim=-1):
        norm = (tensor ** 2).sum(dim=dim, keepdim=True)
        scale = norm / (1 + norm)
        return scale * tensor / torch.sqrt(norm + 1e-8)

    def forward(self, x):
        x = x.unsqueeze(1).unsqueeze(3) @ self.route_weights.unsqueeze(0)
        x = x.squeeze(3)
        b = torch.zeros_like(x[:, :, :, 0], device=x.device)

        for i in range(self.num_iterations):
            c = F.softmax(b, dim=2)
            s = (c.unsqueeze(-1) * x).sum(dim=2)

            # v_j = squash(s_j)
            v = self.squash(s)

            if i < self.num_iterations - 1:
                b = b + (x * v[:, :, None, :]).sum(dim=-1)
        return v

class CapsNet(nn.Module):
    def __init__(self):
        super(CapsNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 256, kernel_size=9, stride=1)
        self.primary_caps = nn.Conv2d(256, 8 * 32, kernel_size=9, stride=2, groups=8)
        self.digit_caps = CapsuleLayer(num_capsules=10, num_route_nodes=32*6*6, in_channels=8, out_channels=16)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.primary_caps(x)
        x = x.view(x.size(0), 32*6*6, 8)
        x = self.digit_caps(x)
        return (x ** 2).sum(dim=-1) ** 0.5

model = CapsNet()
print(model)

CapsNet(
  (conv1): Conv2d(1, 256, kernel_size=(9, 9), stride=(1, 1))
  (primary_caps): Conv2d(256, 256, kernel_size=(9, 9), stride=(2, 2), groups=8)
  (digit_caps): CapsuleLayer()
)


## **Definisi Loss Function (Margin Loss) dan Optimizer**

In [9]:
class MarginLoss(nn.Module):
    def __init__(self, m_plus=0.9, m_minus=0.1, lambda_val=0.5):
        super(MarginLoss, self).__init__()
        self.m_plus = m_plus
        self.m_minus = m_minus
        self.lambda_val = lambda_val

    def forward(self, lengths, target):
        # lengths adalah panjang vektor output (dari CapsNet forward)
        # target adalah label kelas (0, 1, ..., 9)

        # T_k (Indicator function): 1 untuk kelas target, 0 untuk non-target
        T_k = torch.zeros(lengths.size()).to(lengths.device)
        T_k.scatter_(1, target.unsqueeze(1), 1)

        # Loss untuk kelas target: L_k = T_k * max(0, m+ - ||vk||)^2
        loss_pos = T_k * F.relu(self.m_plus - lengths).pow(2)

        # Loss untuk kelas non-target: L_k = lambda * (1 - T_k) * max(0, ||vk|| - m-)^2
        loss_neg = self.lambda_val * (1.0 - T_k) * F.relu(lengths - self.m_minus).pow(2)

        # Total Loss adalah penjumlahan loss di seluruh kapsul
        total_loss = loss_pos + loss_neg
        return total_loss.sum(dim=1).mean()

# Inisialisasi Model, Loss, dan Optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CapsNet().to(device)
criterion = MarginLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

## **Training Loop untuk CapsNet**

In [10]:
# 4. Training Loop
epochs = 10
print(f"\nMemulai Pelatihan CapsNet pada {device}...")

for epoch in range(1, epochs + 1):
    model.train()
    train_loss = 0
    correct = 0
    total = 0

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data) # Output adalah panjang vektor
        loss = criterion(output, target)

        loss.backward()
        optimizer.step()

        train_loss += loss.item()

        # Akurasi: Ambil indeks kapsul dengan panjang vektor terbesar
        _, predicted = torch.max(output.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

        if batch_idx % 100 == 0:
            print(f'Epoch: {epoch} [{batch_idx*len(data)}/{len(train_loader.dataset)}] Loss: {loss.item():.6f}')

    avg_train_loss = train_loss / len(train_loader)
    train_accuracy = 100. * correct / total

    # Evaluasi (Test)
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

    avg_test_loss = test_loss / len(test_loader)
    test_accuracy = 100. * correct / total

    # Hasil
    print(f'\n--- Epoch {epoch}/{epochs} Selesai ---')
    print(f'Loss Pelatihan: {avg_train_loss:.4f}, Akurasi Pelatihan: {train_accuracy:.2f}%')
    print(f'Loss Uji: {avg_test_loss:.4f}, Akurasi Uji: {test_accuracy:.2f}%')
    print('--------------------------------\n')


Memulai Pelatihan CapsNet pada cuda...
Epoch: 1 [0/60000] Loss: 0.806685
Epoch: 1 [6400/60000] Loss: 0.053499
Epoch: 1 [12800/60000] Loss: 0.072911
Epoch: 1 [19200/60000] Loss: 0.022776
Epoch: 1 [25600/60000] Loss: 0.021321
Epoch: 1 [32000/60000] Loss: 0.053397
Epoch: 1 [38400/60000] Loss: 0.053036
Epoch: 1 [44800/60000] Loss: 0.027431
Epoch: 1 [51200/60000] Loss: 0.005660
Epoch: 1 [57600/60000] Loss: 0.021375

--- Epoch 1/10 Selesai ---
Loss Pelatihan: 0.0533, Akurasi Pelatihan: 95.17%
Loss Uji: 0.0204, Akurasi Uji: 98.14%
--------------------------------

Epoch: 2 [0/60000] Loss: 0.008051
Epoch: 2 [6400/60000] Loss: 0.005671
Epoch: 2 [12800/60000] Loss: 0.024167
Epoch: 2 [19200/60000] Loss: 0.022575
Epoch: 2 [25600/60000] Loss: 0.005885
Epoch: 2 [32000/60000] Loss: 0.016280
Epoch: 2 [38400/60000] Loss: 0.009774
Epoch: 2 [44800/60000] Loss: 0.016920
Epoch: 2 [51200/60000] Loss: 0.014089
Epoch: 2 [57600/60000] Loss: 0.034843

--- Epoch 2/10 Selesai ---
Loss Pelatihan: 0.0159, Akurasi 